In [24]:
import numpy as np

In [25]:
with open('models-block-0.txt', 'r') as f:
    fit = f.readlines()

for row in range(len(fit)):
    fit[row] = list(filter(str.strip,fit[row].split(' ')))

In [26]:
#creates dictionary[galaxy id][value/filter name]
#returns photometry or sfr/Mstar for modeled galaxies
galaxy_dict = {}
for row in fit[1:]:
    galaxy_dict[row[0]] = {}
    for i,item in enumerate(row):
        galaxy_dict[row[0]][fit[0][i]] = float(item)

In [27]:
#Dictionary holding upper limits for filters
upper_limits = {}
#LSST & WFIRST limits [AB magnitude -> mJy]
limits = np.array([25.0,27.0,27.0,27.0,25.0,24.4,27.13,27.14,27.12,26.15], dtype=float)
limits = 10**((23.9-limits)/2.5 - 3)
upper_limits['LSST-u'] = limits[0]
upper_limits['LSST-g'] = limits[1]
upper_limits['LSST-r'] = limits[2]
upper_limits['LSST-i'] = limits[3]
upper_limits['LSST-z'] = limits[4]
upper_limits['LSST-y'] = limits[5]
upper_limits['WFIRST_106'] = limits[6]
upper_limits['WFIRST_129'] = limits[7]
upper_limits['WFIRST_158'] = limits[8]
upper_limits['WFIRST_184'] = limits[9]

# 50, 250, & 850um upper limits
upper_limits['IRAS3'] = .0382
upper_limits['PSW'] = .0866
upper_limits['SCUBA850'] = 1.0

In [28]:
lsst_wfirst = ['LSST-u','LSST-g','LSST-r','LSST-i','LSST-z','LSST-y','WFIRST_106','WFIRST_129','WFIRST_158','WFIRST_184']
u50 = ['IRAS3']
u250 = ['PSW']
u850 = ['SCUBA850']

combos = {}
combos['LW'] = lsst_wfirst
combos['LW50'] = lsst_wfirst+u50
combos['LW250'] = lsst_wfirst+u250
combos['LW850'] = lsst_wfirst+u850
combos['LW50250850'] = lsst_wfirst+u50+u250+u850
combos['50250850'] = u50+u250+u850

In [29]:
min_sfr = 1e-3
z = 0.1

# Creat Cigale input for all photometry combos with models above minimum sfr at designated redshift
for filename in combos.keys():
    f = open("input_files/z"+filename+".txt",'w')
    f.write("#id\tredshift\t")
    f.write('\t'.join(combos[filename])+'\n')
    for galaxy in galaxy_dict.keys():
        if( galaxy_dict[galaxy]['sfh.sfr'] >= min_sfr ):
            f.write(galaxy+'\t'+str(z)+'\t')
            for filtername in combos[filename]:
                if( galaxy_dict[galaxy][filtername] >= upper_limits[filtername] ):
                    f.write(str(galaxy_dict[galaxy][filtername])+'\t')
                else:
                    f.write(str(-galaxy_dict[galaxy][filtername])+'\t')
            f.write('\n')
f.close()